## IMF

In [18]:
import seaborn
import imfp
import matplotlib.pyplot as plt 
import pandas as pd 
pd.options.display.max_colwidth=200
imfp.set_imf_wait_time(2)


In [19]:
# Search for desired database
databases = imfp.imf_databases()
databases[databases.description.str.contains("Balance")]

,database_id,description
0,BOP_2017M06,"Balance of Payments (BOP), 2017 M06"
1,BOP_2020M3,"Balance of Payments (BOP), 2020 M03"
2,BOP_2017M11,"Balance of Payments (BOP), 2017 M11"
4,GFSMAB2016,"Government Finance Statistics Yearbook (GFSY 2016), Main Aggregates and Balances"
5,BOP_2019M12,"Balance of Payments (BOP), 2019 M12"
...,...,...
299,BOPAGG,"Balance of Payments (BOP), World and Regional Aggregates"
304,BOP,Balance of Payments (BOP)
315,GFSIBS,"Government Finance Statistics (GFS), Integrated Balance Sheet (Stock Positions and Flows in Assets and Liabilities)"
316,GFSMAB,"Government Finance Statistics (GFS), Main Aggregates and Balances"


In [20]:
# Name of chosen database to use (ID)
database_id = "IFS" #BOP, BOPAGG, BOPSDMXUSD

In [4]:
# Load data set options params: dataset_params via imf_parameters("IFS")
dataset_params = imfp.imf_parameters(database_id) # Load this particular dataset all possible features
dataset_params

{'freq':   input_code description
 0          A      Annual
 1          M     Monthly
 2          Q   Quarterly,
 'ref_area':     input_code                                       description
 0           AF                                       Afghanistan
 1           AL                                           Albania
 2           DZ                                           Algeria
 3           AD                                           Andorra
 4           AO                                            Angola
 ..         ...                                               ...
 266     1C_SRF                                     SRF Countries
 267         F6                                Sub-Saharan Africa
 268         7A  WAEMU (West African Economic and Monetary Union)
 269        A10                                Western Hemisphere
 270        W00                   All Countries, excluding the IO
 
 [271 rows x 2 columns],
 'indicator':              input_code  \
 0            N

In [5]:
# Indicator searcher: search_id.description.str.contains("Gross Capital Formation")
search_id = dataset_params["indicator"]
search_params_particular = search_id[search_id.description.str.contains("Mont")]
search_params_particular

,input_code,description
218,FII_3M_PA,"Financial, Interest Rates, 3-Month Interbank Interest, Percent per annum"
259,FIHN_NFC_L_OT_O1MILL_3M_1Y_PA,"Financial, Interest Rates, Harmonized Euro Area Rates, New Business, Loans, Non-financial Corporations, Other Than Bank Overdrafts, Over EUR 1 Million, Over 3 Months and up to 1 Year, Percent per ..."
269,FILED_3M_PA,"Financial, Interest Rates, London Eurodollar Deposit Rate, Three-Month, Percent per annum"
270,FILIBOR_1M_PA,"Financial, Interest Rates, London Interbank Offer Rate, One-Month, Percent per annum"
273,FILIBOR_6M_PA,"Financial, Interest Rates, London Interbank Offer Rate, Six-Month, Percent per annum"


In [6]:
# Frequency establisher: freq
freq = ["A","Q","M"]

freq = dataset_params["freq"][dataset_params["freq"].input_code.isin(freq)]
freq

,input_code,description
0,A,Annual
1,M,Monthly
2,Q,Quarterly


In [7]:
# Reference area estabilished: ref_area

# Lets go for "ALL" for the moment, later own you might want to reduce the scope
#ref_area_id = ["AF","AL","DZ","AO"]
#ref_area = dataset_params["ref_area"][dataset_params["ref_area"].input_code.isin(ref_area_id)]
ref_area = dataset_params["ref_area"]
display(ref_area)
# Just used to name the variables in a more managable way...
ref_area_swapper = [""]

,input_code,description
0,AF,Afghanistan
1,AL,Albania
2,DZ,Algeria
3,AD,Andorra
4,AO,Angola
...,...,...
266,1C_SRF,SRF Countries
267,F6,Sub-Saharan Africa
268,7A,WAEMU (West African Economic and Monetary Union)
269,A10,Western Hemisphere


In [21]:
# ID params of all indicators we want to search: indicator
indicator_id = ["IAFR_BP6_USD","ILFR_BP6_USD","BCAXF_BP6_USD"]#,"BKAA_BP6_USD","BFXF_BP6_USD","BTRUE_BP6_USD","BEF_BP6_USD","BOPFR_BP6_USD","PXP_IX","PMP_IX","NGDP_SA_XDC"]
indicator = search_id[search_id.input_code.isin(indicator_id)]

# This i will use latter to make the columns of the final output more readable...

# Note: Exceptional financing---> Argentia y alianza latinoamericana
indicator_swapper = ["IIP Net Assets","IIP Net Liabilities","Current Accounbt (Excluding Exceptional Financing)",]#"Capital Account (Excluding Reserves and related items)","Financial Account (Excluding Exceptional Financing)","(BOP)Reserves and realeated items (includes itself Exceptional Financing)","(BOP) Exceptioanl Financing","(BOP)Errors and Omissions","Exports Price Index","Imporrts Price Index","GDP, SA, LC,"]
indicator_swapper_easier = ["IIP(A)","IIP(L)","CA(-Exceptional Financing)"]#,"FA(-Exceptional Financing)","(BOP)Reserves and realted items","(BOP) Exceptional Financing","(BOP) EO","PX Index","PM Index"]
indicator

,input_code,description
6,IAFR_BP6_USD,"International Investment Positions, Net acquisition of financial assets (with Fund Record), US Dollars"
815,ILFR_BP6_USD,"International Investment Positions, Net incurrence of liabilities, US Dollars"
1654,BCAXF_BP6_USD,"Balance of Payments, Supplementary Items, Current Account, Net (excluding exceptional financing), US Dollars"


In [25]:
# Set up the search option that must have the same structure as dataset_params: data_params
data_params = dict(zip(dataset_params.keys(), [[0]] * len(dataset_params.keys())))
data_params["freq"] = freq
data_params["ref_area"] = ref_area
data_params["indicator"] = indicator
data_params

{'freq':   input_code description
 0          A      Annual
 1          M     Monthly
 2          Q   Quarterly,
 'ref_area':     input_code                                       description
 0           AF                                       Afghanistan
 1           AL                                           Albania
 2           DZ                                           Algeria
 3           AD                                           Andorra
 4           AO                                            Angola
 ..         ...                                               ...
 266     1C_SRF                                     SRF Countries
 267         F6                                Sub-Saharan Africa
 268         7A  WAEMU (West African Economic and Monetary Union)
 269        A10                                Western Hemisphere
 270        W00                   All Countries, excluding the IO
 
 [271 rows x 2 columns],
 'indicator':          input_code  \
 6      IAFR_BP6_US

In [26]:
# Get the corresponding data via imf_dataset(database_id,parameters=data_params)
data = imfp.imf_dataset(database_id=database_id, parameters=data_params)


In [27]:
# The output data:
data

,freq,ref_area,indicator,unit_mult,time_format,time_period,obs_value,obs_status
0,A,GH,IAFR_BP6_USD,6,P1Y,2006,3873.02895366812,NaN
1,A,GH,IAFR_BP6_USD,6,P1Y,2007,4673.53802719388,NaN
2,A,GH,IAFR_BP6_USD,6,P1Y,2008,4109.13407935573,NaN
3,A,GH,IAFR_BP6_USD,6,P1Y,2009,5805.13915664204,NaN
4,A,GH,IAFR_BP6_USD,6,P1Y,2010,7616.48274886853,B
...,...,...,...,...,...,...,...,...
19,Q,TN,BCAXF_BP6_USD,6,P3M,2022-Q4,-759.053672024928,NaN
0,A,SM,IAFR_BP6_USD,6,P1Y,2020,6721.24427862693,NaN
1,A,SM,IAFR_BP6_USD,6,P1Y,2021,6757.46734352136,NaN
0,A,SM,ILFR_BP6_USD,6,P1Y,2020,2197.53906700803,NaN


In [28]:
# Now, lets make the output a little bit more readable!!

data_ajusted = data.copy()
# Convert the time_period into time_stamps
def convert_to_datatime(row): # Basically, to each row value of the series data_ajusted["time_period"], apply the transformation
    if "-" in row or "Q" in row:
        return pd.to_datetime(row).strftime('%Y-%m-%d')
    else:
        return pd.to_datetime(row).year
data_ajusted["time_period"] = data_ajusted["time_period"].apply(convert_to_datatime) # Apply the transformation to each row.

# Obs_value to numeric + make the unit mult correct: The unit mult represents the number of zeros to add to the obs_value
data_ajusted["obs_value"] = data_ajusted["obs_value"].astype(float)
data_ajusted["unit_mult"] = data_ajusted["unit_mult"].astype(int)
data_ajusted["obs_value"] = data_ajusted["obs_value"] * 10 ** data_ajusted["unit_mult"]
# Separate data by frequency, then by reference area then by time period
data_ajusted = data_ajusted.pivot_table(
    index=["freq","ref_area","time_period"],
    columns=["indicator"],
    values=["obs_value"]
)

data_ajusted

obs_value                            
indicator                 BCAXF_BP6_USD  IAFR_BP6_USD  ILFR_BP6_USD
freq ref_area time_period                                          
A    1C_355   2011        -8.706128e+08  1.048671e+10  4.854993e+09
              2012        -7.970251e+08  1.197665e+10  5.030704e+09
              2013        -7.468860e+08  1.168237e+10  5.171460e+09
              2014        -6.065598e+08  1.262068e+10  6.530652e+09
              2015        -4.957156e+08  1.196515e+10  6.773588e+09
...                                 ...           ...           ...
Q    ZW       2019-10-01   6.005820e+08           NaN           NaN
              2020-01-01   1.774401e+08           NaN           NaN
              2020-04-01   1.314547e+08           NaN           NaN
              2020-07-01   3.792240e+08           NaN           NaN
              2020-10-01   4.080455e+08           NaN           NaN

[25343 rows x 3 columns]